In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Import dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cobra

from src.gem.yeast8model import Yeast8Model
from src.calc.ablation import get_ablation_ratio
from src.calc.flux import compare_fluxes
from src.calc.growth import get_exch_saturation
from src.viz.bar import ablation_barplot, compare_ablation_times

# Construct models of cells of interest, optimise

In [ ]:
glc_exch_rate = 16.89

wt = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.solution = wt.optimize()

# Saturations

In [ ]:
def find_sat_point(exch_rates, growthrates, tol=1e-8):
    tol_bool = np.abs(np.diff(growthrates)) < tol
    sat_idx = np.argwhere(tol_bool)[0][0]
    sat_point = exch_rates[sat_idx+1]
    return sat_point

## Glucose

Optimise wt with glucose unconstrained

In [ ]:
wt.reset_to_file()
wt.unrestrict_growth()
wt.solution = wt.optimize()

Get saturated glucose uptake:

In [ ]:
wt.solution['r_1714_REV']

> You should get 16.9 mmol g<sub>DW</sub><sup>-1</sup> h<sup>-1</sup>.  This agrees with Elsemman et al. (2022): they predict saturation at 18.6 mmol g<sub>DW</sub><sup>-1</sup> h<sup>-1</sup> and report a range of 16 ~ 19 mmol g<sub>DW</sub><sup>-1</sup> h<sup>-1</sup> from the literature (Blank et al., 2004).

Sweep

In [ ]:
exch_rates = np.linspace(0, 20.0, 100)
growthrates = get_exch_saturation(wt, "r_1714", exch_rates)

Finding saturation point

In [ ]:
sat_point = find_sat_point(exch_rates, growthrates)

In [ ]:
print(sat_point)

In [ ]:
glc_sat_point = sat_point

Growth saturation curve

In [ ]:
fig, ax = plt.subplots()
ax.plot(exch_rates, growthrates)
ax.axvline(x=sat_point, color='tab:blue', linestyle='--', label='Saturation point')
ax.set_xlim((0,20))
ax.legend(loc="lower right")
ax.set_xlabel('Glucose exchange rate ($mmol g_{DW}^{-1} h^{-1}$)')
ax.set_ylabel('Growth rate ($h^{-1}$)')
#ax.set_title('Effect of glucose exchange rate on growth rate')

In [ ]:
# Plot of diff
fig, ax = plt.subplots()
ax.plot(exch_rates[1:], np.diff(growthrates))
ax.axvline(x=sat_point, color='tab:blue', linestyle='--', label='Saturation point')
ax.legend(loc="lower right")
ax.set_yscale('log')
ax.set_xlabel('Glucose exchange rate ($mmol g_{DW}^{-1} h^{-1}$)')
ax.set_ylabel('Difference in growth rate')

## Pyruvate

In [ ]:
wt.reset_to_file()
wt.unrestrict_growth()
wt.add_media_components(['r_2033'])
wt.remove_media_components(['r_1714', 'r_1714_REV'])
wt.solution = wt.optimize()

In [ ]:
wt.solution['r_2033']

> Should get: 8.46

In [ ]:
exch_rates = np.linspace(0, 20.0, 100)
growthrates = get_exch_saturation(wt, "r_2033", exch_rates)

In [ ]:
sat_point = find_sat_point(exch_rates, growthrates)

In [ ]:
print(sat_point)

In [ ]:
pyr_sat_point = sat_point

In [ ]:
fig, ax = plt.subplots()
ax.plot(exch_rates, growthrates)
ax.axvline(x=sat_point, color='tab:blue', linestyle='--', label='Saturation point')
ax.set_xlim((0,20))
ax.legend(loc="lower right")
ax.set_xlabel('Pyruvate exchange rate ($mmol g_{DW}^{-1} h^{-1}$)')
ax.set_ylabel('Growth rate ($h^{-1}$)')
#ax.set_title('Effect of glucose exchange rate on growth rate')

In [ ]:
# Plot of diff
fig, ax = plt.subplots()
ax.plot(exch_rates[1:], np.diff(growthrates))
ax.axvline(x=sat_point, color='tab:blue', linestyle='--', label='Saturation point')
ax.legend(loc="lower right")
ax.set_yscale('log')
ax.set_xlabel('Pyruvate exchange rate ($mmol g_{DW}^{-1} h^{-1}$)')
ax.set_ylabel('Difference in growth rate')

## Ammonium

Different saturation points if glucose or pyruvate is the carbon source.

Glucose as carbon source...

In [ ]:
glc_exch_rate = glc_sat_point # Saturation point, as found earlier

wt = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.add_media_components(['r_1654'])
wt.solution = wt.optimize()

In [ ]:
wt.solution['r_1654']

Sweep.

Note: not removing glucose or else growth doesn't happen at all.  Glucose exchange set to 16.89.

In [ ]:
exch_rates = np.linspace(0, 3.0, 100)
growthrates_glc = get_exch_saturation(wt, "r_1654", exch_rates, remove_glucose=False)

In [ ]:
sat_point_glc = find_sat_point(exch_rates, growthrates_glc)

In [ ]:
print(sat_point_glc)

Pyruvate as carbon source...

In [ ]:
glc_exch_rate = 0
pyr_exch_rate = pyr_sat_point

wt = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
wt.remove_media_components(['r_1714', 'r_1714_REV'])
wt.add_media_components(['r_2033'])
wt.add_media_components(['r_1654'])
wt.model.reactions.get_by_id("r_2033").bounds = (-pyr_exch_rate, 0)
wt.model.reactions.get_by_id("r_2033_REV").bounds = (0, pyr_exch_rate)
wt.solution = wt.optimize()

In [ ]:
growthrates_pyr = get_exch_saturation(wt, "r_1654", exch_rates)

In [ ]:
sat_point_pyr = find_sat_point(exch_rates, growthrates_pyr)

In [ ]:
print(sat_point_pyr)

In [ ]:
fig, ax = plt.subplots()
ax.plot(exch_rates, growthrates_glc, color='tab:blue', label="Growth rate, in glucose")
ax.axvline(x=sat_point_glc, color='tab:blue', linestyle='--', label='Saturation point, in glucose')
ax.plot(exch_rates, growthrates_pyr, color='tab:red', label="Growth rate, in pyruvate")
ax.axvline(x=sat_point_pyr, color='tab:red', linestyle='--', label='Saturation point, in pyruvate')
ax.set_xlim((0,3))
ax.legend(loc="center right")
ax.set_xlabel('Ammonium exchange rate ($mmol g_{DW}^{-1} h^{-1}$)')
ax.set_ylabel('Growth rate ($h^{-1}$)')
#ax.set_title('Effect of glucose exchange rate on growth rate')

In [ ]:
# Plot of diff
fig, ax = plt.subplots()
ax.plot(exch_rates[1:], np.diff(growthrates_glc), color='tab:blue', label="Growth rate, in glucose")
ax.axvline(x=sat_point_glc, color='tab:blue', linestyle='--', label='Saturation point, in glucose')
ax.plot(exch_rates[1:], np.diff(growthrates_pyr), color='tab:red', label="Growth rate, in pyruvate")
ax.axvline(x=sat_point_pyr, color='tab:red', linestyle='--', label='Saturation point, in pyruvate')
ax.legend(loc="center right")
ax.set_yscale('log')
ax.set_xlabel('Ammonium exchange rate ($mmol g_{DW}^{-1} h^{-1}$)')
ax.set_ylabel('Difference in growth rate')

# Interpolate

In [ ]:
np.interp(0.1, growthrate, exch_rates)